In [2]:
from keras.models import load_model
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from keras.preprocessing.sequence import pad_sequences

In [3]:
clawing_model = load_model('cnn&rnn_model.h5')

dict_fileWord=pd.read_csv('dict_file.csv')

# test_documents = pd.read_csv ('non_labelData.csv')
# test_documents

In [4]:
def getinput(train):
    list_word = []
    proc = word_tokenize(train, engine='newmm')
    list_word.append(proc) #ตัดคำใหม่ที่รับเข้ามาเสร็จใส่list
    
    sequence = list()
    for n in list_word: #ลูปเปลี่ยน คำ string ที่ตัดมาแปลงเป็นตัวเลข
        sequenceTemp = list() 
        for w in n:
            count=0
            for x in dict_fileWord['Unnamed: 0']: #ลูปเทียบdictคำ
                count=count+1
                if x==w:
                    sequenceTemp.append(count)
                    break
        sequence.append(sequenceTemp)  
    test = np.array(sequence)

    testX = pad_sequences(test, maxlen=105, padding="post") #input_length=105
    return testX

In [5]:
# แบบกรอกทีละประโยค


def normalizeAndPredict_v2(data):
    # กระบวนการหาผลลัพธ์ เตรียมใส label อัตโนมัติ
#     print(data['word'])
        label_toxic1=[]
        label_toxic2=[]
        label_toxic3=[]
        list_label_toxic = [] #ตำแหน่งที่เป็น multilabel

        test=getinput(data)
    
        ynew = clawing_model.predict(test)
        yn = []

        for b in ynew[0]:
            num_per = '%.4f' % b
            yn.append(num_per)
        max_ynew = max(ynew[0][:])
        print('max :-- ','%.4f' % max_ynew)
        tenPer_max_ynew = max_ynew * 0.9
        print('tenper --','%.4f' % tenPer_max_ynew)
        print('result_predict :====',yn)
#                                                                                      passed


        list_label=[] #เก็บค่าที่เป็น Multilabel ไว้ที่นี้
        for i in yn:
    #         print('multilabel :++++++++++++',i)
            if(i >= '%.4f' %tenPer_max_ynew ):
                list_label.append(i)
        print('ผลลัพธ์จากการทำนาย Multilabel ::::::::::',list_label)                                 #passed 

    #     เก็บตำแหน่งที่เป็น multilabel โดยการเทียบกับ yn
        for ii in list_label:
            count = 0 
            for iii in yn:
                count+=1
                if(ii == iii):
                    list_label_toxic.append(count)
                    break
        print('ตำแหน่งของ Multilabel จากที่ทำนาย ::::::::::',list_label_toxic)                      #passed 

        count_j=0
        count_len=0
        for j in list_label_toxic:# มีขนาดตามตำแหน่งที่รับมา
            count_j+=1

            if(count_j==1):
                print('Index_multilabel::::::::::',list_label_toxic[0],'lenght :::: 1')
                label_toxic1.append(list_label_toxic[0])
                count_len=1
                continue

            if(count_j==2):
                print('Index_multilabel::::::::::',list_label_toxic[0:2],'lenght :::: 2')
                label_toxic2.append(list_label_toxic[1])
                count_len=2
                continue

            if(count_j==3):
                print('Index_multilabel::::::::::',list_label_toxic[0:3],'lenght :::: 3')
                label_toxic3.append(list_label_toxic[2])
                count_len=3
                continue
                                                                                                                            #passed 
    # แทนที่ตำแหน่งที่ไม่มีด้วย 0
        if( count_len==1):
            label_toxic2.append(0)
            label_toxic3.append(0)
        if( count_len==2):
            label_toxic3.append(0)
    #    ตรวจตำแหน่งอีกรอบด้วยตาตามแนวตั้ง     
#         print(label_toxic1,label_toxic2,label_toxic3) 
        return [data,label_toxic1,label_toxic2,label_toxic3,list_label_toxic]
        

In [6]:
def to_csv_v2(predict_result):
    
       # read previous csv file before append the new word
        old_csv = pd.read_csv('predict_result.csv');
        if(len(old_csv ) !=0 ):
            list_word = []
            t_all = []
            for word in old_csv['word']:
                list_word.append(word)
            for tword in old_csv['t_class']:
                t_all.append(tword)
        else:
           # only first time to create csv file    
            list_word = []
            t_all = []


        # นำdata ที่ผ่านการ normal มาวนตำแหน่ง 1-3 เพื่อเอาผลการแยกประเภท
        for i in range(1,4):
            if(predict_result[i][0]!=0): #ถ้าไม่เท่ากัยศูนย์แสดงว่ามีการใส่เลขคลาสให้ เช่น 3 0 0 เราก็เอามาแค่อันแรกพอ
                t_all.append(predict_result[i][0])
                list_word.append(predict_result[0])
                
#         print(list_word)
#         print(t_all)
        df = pd.DataFrame({'word':list_word
                           ,'t_class':t_all})
        df.to_csv('predict_result.csv',index=False,encoding='utf-8-sig')

        

In [7]:
predict_result = pd.read_csv('predict_result.csv')
predict_result

,word,t_class
0,ควยพ่องมึงตายไอสัส,3
1,หน้าหนังหี,5
2,อยากตายหรอ,3
3,อยากตายหรอ,6
4,เป็นเอ๋ออ่อ,5
...,...,...
2900,#สันด้านคนมันเหี้ยยังงัยมันก้อปิดไม่สนิทหลอก!!...,4
2901,ทรงกระบอกที่ว่าเหี้ยก็ยังสู้อานม้าไม่ได้ #ควยย...,5
2902,ขอบคุณที่อยู่เป็นเพื่อนนะไอสัส \n#เสียงดังอยู่...,4
2903,ไฟดับ ฝนไม่ตก คลื่นดีแทค หลุม \n\n#ควยย😂,3


In [8]:
from flask import Flask,render_template,request,url_for,jsonify

In [9]:
#  predict แล้วเก็บจากทั้งที่ถูก และ คนทำให้ถูก
app = Flask(__name__)

  
@app.route('/')
def index():
    return render_template('ver3/index.php')
@app.route('/check') 
def check():
    args = request.args
    bword = args['bw'] # get word from web by string query
    global result
    result = normalizeAndPredict_v2(bword) # predict word from get
#     result[4] = [2,5,4] 
    limit_result = []
    if(len(result[4])<3):
        
            for i in range(len(result[4])):
                if(result[4][i]!=None):
                    limit_result.append(result[4][i])
    else:
        
            for i in range(3):
                if(result[4][i]!=None):
                    limit_result.append(result[4][i])

    # ต้องเช็คความยาวของ array ก่อนส่งด้วยห้ามเกิน 3 หรือ loop เอาแค่ 3 อันพอก็ได้
    return render_template('ver3/check.php',show_bword=bword,index=limit_result)

@app.route('/save')
def save():
    args = request.args
    tmp = args['action']
    if(tmp == '1'):
        print('this is a one')
        to_csv_v2(result)
        print(result,'all result action 1')

    elif(tmp == '2'):
        print('this is a two')
        manual = []
        for i in range(1,7):
            try:
                class_from_query_string = args['bclass'+str(i)]
                print("well done",class_from_query_string)
                manual.append(class_from_query_string)
                
            except:
                print('it dont have on it',i)
        print('this is manual list',manual)
        mlb1 = []
        mlb2 = []
        mlb3 = []
        print(mlb1,mlb2,mlb3)
        
        if(len(manual)==1):
            mlb1.append(int(manual[0]))
            mlb2.append(0)
            mlb3.append(0)
            print('len 1')
            print(mlb1,mlb2,mlb3)
            
        elif(len(manual)==2):
            mlb1.append(int(manual[0]))
            mlb2.append(int(manual[1]))
            mlb3.append(0)
            print('len 2')
            print(mlb1,mlb2,mlb3)
        else:
            mlb1.append(int(manual[0]))
            mlb2.append(int(manual[1]))
            mlb3.append(int(manual[2]))
            print('len 3')
            print(mlb1,mlb2,mlb3)
#         print('this are mlb1 mlb2 and mlb3 by order : ',mlb1,mlb2,mlb3)

        print(result,'all in result')
    
        result[1][0] = mlb1[0]
        result[2][0] = mlb2[0]
        result[3][0] = mlb3[0]
        
        print(result,'all in new result')
        
        to_csv_v2(result)
    return render_template('ver3/success.php')  
    

@app.route('/notsave')
def notsave():

    args = request.args
    bword = args['bw']
#     print(result[4][0])
    return render_template('ver3/manual.php',show_bword=bword)


if __name__ == '__main__':
   app.run(port=80)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2020 22:11:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 22:11:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2020 22:11:49] "GET /check?bw=asdasdasd HTTP/1.1" 200 -


max :--  0.6721
tenper -- 0.6049
result_predict :==== ['0.6721', '0.0659', '0.0136', '0.0016', '0.1929', '0.0539']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6721']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1


127.0.0.1 - - [01/Nov/2020 22:11:50] "GET /save?action=1&bw=asdasdasd HTTP/1.1" 200 -


this is a one
['asdasdasd', [1], [0], [0], [1]] all result action 1


127.0.0.1 - - [01/Nov/2020 22:11:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 22:11:53] "GET /check?bw=asdfasfsadf HTTP/1.1" 200 -


max :--  0.6721
tenper -- 0.6049
result_predict :==== ['0.6721', '0.0659', '0.0136', '0.0016', '0.1929', '0.0539']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6721']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1


127.0.0.1 - - [01/Nov/2020 22:11:54] "GET /notsave?bw=asdfasfsadf&index=[1] HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 22:11:55] "GET /save?action=2&bclass1=1 HTTP/1.1" 200 -


this is a two
well done 1
it dont have on it 2
it dont have on it 3
it dont have on it 4
it dont have on it 5
it dont have on it 6
this is manual list ['1']
[] [] []
len 1
[1] [0] [0]
['asdfasfsadf', [1], [0], [0], [1]] all in result
['asdfasfsadf', [1], [0], [0], [1]] all in new result


127.0.0.1 - - [01/Nov/2020 22:11:56] "GET / HTTP/1.1" 200 -


## # สำหรับ auto label ที่ละหลายประโยค

In [10]:
def to_multi_autolabel(path):
    data = pd.read_csv(path)
    Fromlist_scraper = []
    for w in data['word']:
        Fromlist_scraper = normalizeAndPredict_v2(w)
#         print(Fromlist_scraper)
        to_csv_v2(Fromlist_scraper)
    return 'label complete see data at predict_result.csv file'

In [11]:
p = './dataset/dataset_tweet.csv'
test = to_multi_autolabel(p)

max :--  0.8858
tenper -- 0.7972
result_predict :==== ['0.0000', '0.0743', '0.0000', '0.8858', '0.0399', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.8858']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_multilabel:::::::::: 4 lenght :::: 1
max :--  0.6721
tenper -- 0.6049
result_predict :==== ['0.6721', '0.0659', '0.0136', '0.0016', '0.1929', '0.0539']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6721']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1
max :--  0.8938
tenper -- 0.8044
result_predict :==== ['0.0002', '0.8938', '0.0000', '0.0379', '0.0681', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.8938']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  0.9982
tenper -- 0.8984
result_predict :==== ['0.0000', '0.0018', '0.0000', '0.9982', '0.0000', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9982']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_m

max :--  0.9844
tenper -- 0.8859
result_predict :==== ['0.0000', '0.0000', '0.0022', '0.9844', '0.0012', '0.0122']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9844']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_multilabel:::::::::: 4 lenght :::: 1
max :--  0.5517
tenper -- 0.4965
result_predict :==== ['0.0000', '0.0419', '0.0008', '0.4055', '0.5517', '0.0001']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5517']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_multilabel:::::::::: 5 lenght :::: 1
max :--  0.5207
tenper -- 0.4686
result_predict :==== ['0.0000', '0.0000', '0.4775', '0.5207', '0.0013', '0.0006']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.4775', '0.5207']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [3, 4]
Index_multilabel:::::::::: 3 lenght :::: 1
Index_multilabel:::::::::: [3, 4] lenght :::: 2
max :--  0.5202
tenper -- 0.4682
result_predict :==== ['0.0000', '0.0000', '0.0125', '0.4672', '0.5202', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.52

max :--  0.5077
tenper -- 0.4569
result_predict :==== ['0.0000', '0.0000', '0.5077', '0.4316', '0.0060', '0.0547']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5077']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [3]
Index_multilabel:::::::::: 3 lenght :::: 1
max :--  0.4458
tenper -- 0.4013
result_predict :==== ['0.0071', '0.2289', '0.0291', '0.0067', '0.2824', '0.4458']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.4458']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [6]
Index_multilabel:::::::::: 6 lenght :::: 1
max :--  0.6105
tenper -- 0.5494
result_predict :==== ['0.0000', '0.6105', '0.0000', '0.3282', '0.0613', '0.0001']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6105']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  0.9701
tenper -- 0.8731
result_predict :==== ['0.0000', '0.9701', '0.0000', '0.0016', '0.0283', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9701']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_m

max :--  0.5302
tenper -- 0.4772
result_predict :==== ['0.0000', '0.5302', '0.0000', '0.0068', '0.4630', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5302']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  0.9104
tenper -- 0.8193
result_predict :==== ['0.0000', '0.0028', '0.0000', '0.9104', '0.0869', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9104']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_multilabel:::::::::: 4 lenght :::: 1
max :--  0.7145
tenper -- 0.6431
result_predict :==== ['0.7145', '0.0605', '0.0129', '0.0014', '0.1640', '0.0466']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.7145']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1
max :--  0.5865
tenper -- 0.5279
result_predict :==== ['0.0576', '0.3220', '0.0025', '0.0011', '0.5865', '0.0304']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5865']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_m

max :--  0.9811
tenper -- 0.8830
result_predict :==== ['0.0000', '0.9811', '0.0000', '0.0120', '0.0068', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9811']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  0.8381
tenper -- 0.7543
result_predict :==== ['0.0000', '0.0003', '0.0000', '0.8381', '0.1616', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.8381']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_multilabel:::::::::: 4 lenght :::: 1
max :--  0.6256
tenper -- 0.5631
result_predict :==== ['0.0000', '0.3441', '0.0000', '0.6256', '0.0302', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6256']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_multilabel:::::::::: 4 lenght :::: 1
max :--  0.9387
tenper -- 0.8448
result_predict :==== ['0.0000', '0.9387', '0.0000', '0.0465', '0.0147', '0.0001']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9387']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_m

...

max :--  0.7980
tenper -- 0.7182
result_predict :==== ['0.7980', '0.0014', '0.1612', '0.0142', '0.0188', '0.0065']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.7980']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1
max :--  0.7434
tenper -- 0.6691
result_predict :==== ['0.0000', '0.7434', '0.0000', '0.0239', '0.2326', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.7434']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  0.5108
tenper -- 0.4597
result_predict :==== ['0.0014', '0.0272', '0.1347', '0.3123', '0.5108', '0.0137']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5108']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_multilabel:::::::::: 5 lenght :::: 1
max :--  0.5379
tenper -- 0.4841
result_predict :==== ['0.3406', '0.5379', '0.0120', '0.0119', '0.0847', '0.0129']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5379']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_m

max :--  0.6405
tenper -- 0.5764
result_predict :==== ['0.0883', '0.0126', '0.6405', '0.0142', '0.0685', '0.1759']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6405']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [3]
Index_multilabel:::::::::: 3 lenght :::: 1
max :--  0.9999
tenper -- 0.8999
result_predict :==== ['0.9999', '0.0000', '0.0001', '0.0000', '0.0000', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9999']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1
max :--  0.8113
tenper -- 0.7302
result_predict :==== ['0.1867', '0.8113', '0.0001', '0.0008', '0.0011', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.8113']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  0.5624
tenper -- 0.5062
result_predict :==== ['0.0004', '0.5624', '0.0003', '0.1382', '0.2827', '0.0160']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5624']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_m

max :--  0.3914
tenper -- 0.3523
result_predict :==== ['0.0003', '0.3081', '0.0002', '0.2953', '0.3914', '0.0048']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.3914']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_multilabel:::::::::: 5 lenght :::: 1
max :--  0.5427
tenper -- 0.4885
result_predict :==== ['0.0020', '0.0893', '0.0069', '0.0262', '0.5427', '0.3329']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5427']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_multilabel:::::::::: 5 lenght :::: 1
max :--  0.4189
tenper -- 0.3770
result_predict :==== ['0.0000', '0.3915', '0.0000', '0.1829', '0.4189', '0.0067']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.3915', '0.4189']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2, 5]
Index_multilabel:::::::::: 2 lenght :::: 1
Index_multilabel:::::::::: [2, 5] lenght :::: 2
max :--  0.9197
tenper -- 0.8277
result_predict :==== ['0.0027', '0.0311', '0.0091', '0.9197', '0.0362', '0.0013']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.91

max :--  0.6721
tenper -- 0.6049
result_predict :==== ['0.6721', '0.0659', '0.0136', '0.0016', '0.1929', '0.0539']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.6721']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1
max :--  0.9504
tenper -- 0.8554
result_predict :==== ['0.0000', '0.0067', '0.0000', '0.9504', '0.0428', '0.0001']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9504']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_multilabel:::::::::: 4 lenght :::: 1
max :--  0.9043
tenper -- 0.8139
result_predict :==== ['0.0001', '0.0001', '0.9043', '0.0210', '0.0055', '0.0691']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.9043']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [3]
Index_multilabel:::::::::: 3 lenght :::: 1
max :--  0.4786
tenper -- 0.4307
result_predict :==== ['0.0004', '0.4270', '0.0004', '0.0535', '0.4786', '0.0402']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.4786']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_m

max :--  0.7144
tenper -- 0.6430
result_predict :==== ['0.0000', '0.2605', '0.0000', '0.0122', '0.7144', '0.0128']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.7144']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [5]
Index_multilabel:::::::::: 5 lenght :::: 1
max :--  0.3772
tenper -- 0.3395
result_predict :==== ['0.0000', '0.3772', '0.0000', '0.3266', '0.2960', '0.0001']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.3772']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [2]
Index_multilabel:::::::::: 2 lenght :::: 1
max :--  1.0000
tenper -- 0.9000
result_predict :==== ['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['1.0000']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [1]
Index_multilabel:::::::::: 1 lenght :::: 1
max :--  0.5320
tenper -- 0.4788
result_predict :==== ['0.0000', '0.0004', '0.4043', '0.5320', '0.0099', '0.0533']
ผลลัพธ์จากการทำนาย Multilabel :::::::::: ['0.5320']
ตำแหน่งของ Multilabel จากที่ทำนาย :::::::::: [4]
Index_m

In [12]:
test

'label complete see data at predict_result.csv file'